# OBJECTIVE

## Using different neural network topologies, predict oil production in the province of Neuquen, Argentina within the next few years alongside crude oil prices, obtaining an estimation of the corresponding economic dividends.

In [7]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
import tensorflow as tf
from fredapi import Fred

# Define some colors for later on
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# FRED API

The proper functioning of this notebook requires use of the freely-available FRED API. The user must supply their own API key. The user may supply their own key via an *API_key.txt* file in the same working directory of this notebook, or supply it after running the following code block.

## FRED API keys freely available at https://fred.stlouisfed.org/docs/api/api_key.html

In [8]:
# Try reading FRED API key file

keyfile = False
try:
    with open('FRED_API_KEY', 'r') as file:
        fred_key = file.read().strip()
    print(color.GREEN + color.BOLD + '\n===== FRED API KEY FOUND! =====' + color.END)
    keyfile = True
except:
    print(color.BOLD + color.RED + '===== FRED API KEY NOT FOUND! =====' + color.END)
    fred_key = input('Please supply user FRED API key:')

# Test FRED API key
print('\nTesting FRED API key...')
fred = Fred(api_key=fred_key)
print('Grabbing US GDP data...\n')
try:
    test_data = fred.get_series_first_release('GDP')
    print(test_data)
    print(color.GREEN + color.BOLD + '\n===== API key good, FRED server reachable! =====' + color.END)
except:
    print(color.RED + color.BOLD + '====== TEST FAILED! FRED API key invalid and/or FRED server unreachable (check internet connection). =====' + color.END)
    raise Exception()
    
# Offer to save FRED API in key 
while(not keyfile):
    save_prompt = input('\nSave FRED API key to current working directory for future use? y/[n]').lower()
    if (save_prompt == 'y'):
        try:
            print(fred_key,  file=open('FRED_API_KEY', 'w'))
            print(color.GREEN + 'API key saved to file FRED_API_KEY in current directory!' + color.END)
            break
        except:
            print(color.RED + color.BOLD + '====== COULD NOT WRITE API KEY TO FILE! Check current directory and/or file permissions. =====' + color.END)
    elif (save_prompt == 'n' or save_prompt == ''):
        break
    else:
        print(color.RED + 'Invalid character input. Try again.' + color.END)


===== FRED API KEY FOUND! =====

Testing FRED API key...
Grabbing US GDP data...

date
1946-01-01        199.7
1946-04-01        207.7
1946-07-01        217.9
1946-10-01        222.2
1947-01-01        226.7
                ...    
2022-10-01    26132.458
2023-01-01    26465.865
2023-04-01    26834.953
2023-07-01    27623.543
2023-10-01    27938.831
Name: value, Length: 312, dtype: object

===== API key good, FRED server reachable! =====


In [9]:
# Load CSV data 
nqn_pet = pd.read_csv('C:/Users/Felipe/Desktop/Data_Science_Road_2024/EDA/serie_tiempo_neuquen.csv')

nqn_pet['Produccion_Total_Petroleo']=nqn_pet['Produccion_Total_Petroleo']*(6.28981/1e6) #barriles de petroleo

len(nqn_pet)

nqn_pet=nqn_pet[['fecha','Produccion_Total_Petroleo']].copy()

# Convertir la columna 'fecha' a tipo datetime si aún no lo está
nqn_pet['fecha'] = pd.to_datetime(nqn_pet['fecha'])

# Establecer la columna 'fecha' como el índice
nqn_pet.set_index('fecha', inplace=True)

print(nqn_pet)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Felipe/Desktop/Data_Science_Road_2024/EDA/serie_tiempo_neuquen.csv'

In [ ]:
#SEARCH FOR CRUDE OIL PRICE

In [ ]:
fred = Fred(api_key=fred_key)

In [ ]:
Crude_oil=fred.search("crude oil prices", order_by="popularity")

In [ ]:
Crude_oil.head(10)

In [ ]:
CRUDE=fred.get_series(series_id="WTISPLC")

#CRUDE=CRUDE[['TIME', 'VALUE']]
#CRUDE.columns= ['TIME', 'VALUE'].copy()

#CRUDE = pd.to_datetime(CRUDE)

CRUDE_2010 = CRUDE[CRUDE.index.year >= 2010]

CRUDE_2010=CRUDE_2010.iloc[:-1]
print(CRUDE_2010)
print(nqn_pet)

In [ ]:
SMD=CRUDE_2010*nqn_pet["Produccion_Total_Petroleo"]

# Crear una figura y un eje
fig, ax = plt.subplots(figsize=(10, 6))

# Graficar la serie de tiempo
ax.plot(SMD.index, SMD.values, marker='o', linestyle='-', color='b', label='Producción de Petróleo en Millones de Dólares por Barril')

# Configurar el eje x para mostrar solo años
ax.set_xticks(SMD.resample('Y').mean().index)
ax.set_xticklabels([str(year.year) for year in SMD.resample('Y').mean().index], rotation=45)

# Etiquetas y título
ax.set_xlabel('Año')
ax.set_ylabel('Producción de Petróleo (Millones de Dólares por Barril de Crudo en Neuquen)')
ax.set_title('Producción de Petróleo en Millones de Dólares por Barril a lo largo del Tiempo')

# Leyenda
ax.legend()

# Mostrar el gráfico
plt.show()

# Predicciones

In [ ]:
#vamos a normalizar las dos series de tiempo y luego dividir en dos set de datos 70% para entrenamiento y 30% para testeo
from sklearn.preprocessing import MinMaxScaler


nqn= nqn_pet["Produccion_Total_Petroleo"].values.reshape(-1, 1)  # Reshape to a 2D array for MinMaxScaler



# Creamos el objeto MinMaxScaler
scaler_min_max_scaler = MinMaxScaler()

# Normalizamos los datos y se agrega como nueva columna

nqn=scaler_min_max_scaler.fit_transform(nqn)



nqn_pet["Produccion_Total_Petroleo"] = nqn.flatten()


nqn_pet.head()

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop


seq_length=12 #anual
# Construir el modelo de red neuronal
#model = tf.keras.Sequential([
 #   tf.keras.layers.LSTM(32,activation="relu", input_shape=(seq_length, 1)),
  #  tf.keras.layers.LSTM(64, activation="relu"),
   # tf.keras.layers.Dense(1, activation="softmax")
#])


#model = Sequential()
#model.add(layers.LSTM(32, dropout=0.1, recurrent_dropout=0.5,
                #     return_sequences=True, input_shape=(seq_length, 1)))

#model.add(layers.LSTM(64, activation='relu',
                 #    dropout=0.1, recurrent_dropout=0.5))

#model.add(layers.Dense(1))




model = Sequential()
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5,
                     return_sequences=True, input_shape=(seq_length, 1)))

model.add(layers.GRU(64, activation='relu',
                     dropout=0.1, recurrent_dropout=0.5, return_sequences=True))

model.add(layers.GRU(128, activation='relu',
                     dropout=0.1, recurrent_dropout=0.5))


model.add(layers.Dense(1))








# Compilar el modelo'
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse','acc' ])

# Resumen del modelo
model.summary()



In [ ]:
# Definimos el tamaño del conjunto de entrenamiento (por ejemplo, el 70% de los datos)
train_size = int(len(nqn) * 0.6)

# Dividimos los datos en conjuntos de entrenamiento y prueba de manera secuencial
train_data, test_data = nqn[:train_size], nqn[train_size:]


seq_length= 12  # ajusta este valor según tus necesidades

# Crear las secuencias de entrada y salida para entrenamiento
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)


# Convierte a arrays numpy
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)


In [ ]:
X_train.shape


In [ ]:
X = np.array(X_train).reshape(-1, seq_length, 1)
y = np.array(y_train)

# Imprimir la forma de los datos de entrada
print("Forma de X:", X.shape)  # Salida esperada: (número de secuencias, longitud de secuencia, número de características)
print("Forma de y:", y.shape)  # Salida esperada: (número de sec



#Go_train


model.fit(X_train, y_train, epochs=15)






In [ ]:


predictions_t=model.predict(X_train)
predictions_t=scaler_min_max_scaler.inverse_transform(predictions_t)
y_train = scaler_min_max_scaler.inverse_transform(y_train)


# Graficar los resultados
plt.figure(figsize=(10, 6))

# Graficar los valores reales
plt.plot(y_train, label='Valores Reales', marker='o')

# Graficar las predicciones
plt.plot(predictions_t, label='Predicciones', marker='x')

plt.title('Comparación entre Valores Reales y Predicciones')
plt.xlabel('Índice de la Muestra')
plt.ylabel('Valor')
plt.legend()
plt.show()




In [ ]:
# Realizar predicciones con el modelo entrenado
predictions = model.predict(X_test)

predictions = scaler_min_max_scaler.inverse_transform(predictions)
y_test = scaler_min_max_scaler .inverse_transform(y_test)



# Graficar los resultados
plt.figure(figsize=(10, 6))

# Graficar los valores reales
plt.plot(y_test, label='Valores Reales', marker='o')

# Graficar las predicciones
plt.plot(predictions, label='Predicciones', marker='x')

plt.title('Comparación entre Valores Reales y Predicciones')
plt.xlabel('Índice de la Muestra')
plt.ylabel('Valor')
plt.legend()
plt.show()



In [ ]:

def graph_real_pred(y_test, y_pred_train, color, title, ax=None):
    """
    Funcion que grafica los valores reales vs. predichos
    :param y_test: valores reales
    :param y_pred_train: valores predichos
    :param color: color para el plot.
    :param title: título del gráfico
    :param ax: objeto de ejes de Matplotlib para dibujar el gráfico (opcional)
    :return: Scatterplot mostrando la relación entre el valor real y el predicho
    """
    if ax is None:
        ax = plt.gca()
    
    ax.scatter(y_test, y_pred_train, c=color, s=10)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlabel('Real', size=10, labelpad=1)
    ax.set_ylabel('Predicted', size=10, labelpad=1)
    ax.set_title(title, size=18)  # Agregamos título al gráfico




graph_real_pred(y_test, predictions,"blue","LSTM32,63 con dropout" )

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R^2 Score: {r2}')

Un r2 negativo significa que el modelo es realmente inapropiado para los datos y que las predicciones son peores que simplemente usar la media de los valores observados como predicción. Esto puede ocurrir cuando el modelo tiene un rendimiento muy deficiente y está realizando predicciones extremadamente alejadas de los valores reales.

In [ ]:
En primer lugar probé con una LSTM con 32 unidades (Fig. 1).

En segundo lugar uni dos LSTM con 32 y 64 unidades respectivamente. El resultado mejoró notablemente, porque lo que se buscaba era que siguiera la tendencia alsista (Fig. 2).

Asi que aumenté la profundidad agregando otra capa de 128 unidades y estos fueron los resultados (Fig. 3).


Combine una Conv1D con Maxpooling y una GRU 32 unidades (Fig. 4).

Y por último agregue una capa más GRU 64 a ver si mejoraba el resultado (Fig 5).


In [ ]:
# Define future dates for the next 5 years
future_dates = pd.date_range(start=nqn_pet.index[-1] + pd.Timedelta(days=30),  periods=3*12, freq='M')

# Prepare the last sequence of scaled closing prices as input for prediction
future_seq = nqn[-seq_length:].reshape(1, seq_length, 1)

# Make predictions for the next 5 years
future_pred = []

for _ in range(3*12):
    # Predict the next value using the LSTM model
    next_pred = model.predict(future_seq)[0][0]
    
    # Append the predicted value to the list of future predictions
    future_pred.append(next_pred)
    
    # Update the input sequence for the next prediction
    future_seq = np.append(future_seq[:, 1:, :], [[[next_pred]]], axis=1)

# Inverse transform the predicted values back to the original scale
future_pred = scaler_min_max_scaler.inverse_transform(np.array(future_pred).reshape(-1, 1))



In [ ]:
# Gráfico para la proyección futura


plt.figure(figsize=(12, 18))


plt.subplot(3, 1, 3)
plt.plot(nqn_pet, label='Datos históricos')
plt.plot(future_dates, future_pred[:len(future_dates)], linestyle='--', label='Proyección futura (3 años)')
plt.title('Proyección de comportamiento a futuro de la produccion')
plt.xlabel('Fecha')
plt.ylabel('Produccion de petroleo')
plt.legend()
plt.grid(True)

